# IMT.fi Customer Segments comparison 2019-2024

Olli Salli, 2025

Note! A typical notebook would have commentary and more informative headings; these are OMITTED HERE to avoid leaking business details to those without access to the source data. 

In [ ]:
start_date = '2019-01-01'
end_date = '2024-12-31'

In [ ]:
from db_connection import configure_pandas_display, get_redshift_engine

configure_pandas_display()

engine = get_redshift_engine('imt_aws_redshift', 'dev')

%load_ext sql
%sql engine
%config SqlMagic.autopandas = True
%config SqlMagic.named_parameters = 'enabled'

## Exploration of individual dimensions

### Biographic

#### Early-stage reservations, broad categories

In [ ]:
%%sql --save passgroup_categories_19_24
select
    groupcategory,

    sum(case when departureyear = 2019 then 1 else 0 end) as res_2019,
    sum(case when departureyear = 2024 then 1 else 0 end) as res_2024,

    sum(case when departureyear = 2019 and finalstage = 'Confirmed' then 1 else 0 end) as confirmed_2019,
    sum(case when departureyear = 2024 and finalstage = 'Confirmed' then 1 else 0 end) as confirmed_2024

from dbt_osalli.fct_reservation_funnel
inner join dbt_osalli.dim_passengergroup using (reservationid, departureyear)

where departuredate between :start_date and :end_date
and departureyear in (2019, 2024)

# Only reservations past the departure step, because only there are the passenger counts and adult/child
# defined for many reservations; we would have an overabundance of default 2-adult reservations otherwise
and funnel_progress > 0

group by groupcategory

order by 1

#### Later-stage, with exact ages known, more specific categories

In [ ]:
%%sql --save passgroup_adultage_categories_19_24
select
    groupcategory,
    adultagecategory,

    sum(case when departureyear = 2019 then 1 else 0 end) as res_2019,
    sum(case when departureyear = 2024 then 1 else 0 end) as res_2024,

    sum(case when departureyear = 2019 and finalstage = 'Confirmed' then 1 else 0 end) as confirmed_2019,
    sum(case when departureyear = 2024 and finalstage = 'Confirmed' then 1 else 0 end) as confirmed_2024

from dbt_osalli.fct_reservation_funnel
inner join dbt_osalli.dim_passengergroup using (reservationid, departureyear)

where departuredate between :start_date and :end_date
and departureyear in (2019, 2024)

# Passenger info fully filled or then onwards, or data from previous reservations picked through customer account or internal salesperson
and allagesknown

group by groupcategory, adultagecategory

order by 1, 2